In [1]:
import facebook
import requests

token = 'EAACEdEose0cBAHvajjqWOZBwArJ5JQG1KjptGDXYCBGb9ukppJw2Tw5XlJnHWM3ztmQFAw110JlU2aDQvRCREUs5JQLZAbcVaUZBAkWuPoxeLqsx6ux4pVRaeGhED5nloCMeZAbYidemoqOGZACYZAOzFNMOCZCUZCNetbJ23ly1OyZAZBe6diTcHuXgfTfM8OxpBtewKm2uifEQZDZD'
graph = facebook.GraphAPI(access_token = token)

In [2]:
facebookid = '100009402784677'

In [3]:
pic = ',picture.height(720).width(720){url}'
lim = '.limit(5)'
lim1 = '.limit(1)'

In [4]:
def getMyPost(graph):
    event = graph.request('me?fields=name,posts'+lim1+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['posts']['data']:
        try:
            data = {
                'message': d['message'],
                'time': d['created_time']
            }
            ret.append(data)
        except:
            ret = ret
    return {'name': name, 'pic': picture, 'data':ret}

import requests
def getMyPostLike(graph):
    event = graph.request('me?fields=name,posts'+lim1+'{message,likes}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['posts']['data']:
        data = {
            'message': d['message'],
            'count':0,
        }
        data['count']+= len(d['likes']['data'])
        try:
            while(true):
                url = d['likes']['paging']['next']
                resp = requests.get(url)
                d['likes'] = resp.json()
                data['count'] += len(d['likes']['data'])
        except:
            data['count']+=0
        ret.append(data)
        
    return {'name': name, 'pic': picture, 'data':ret}

def getMyPostComment(graph):
    event = graph.request('me?fields=name,posts'+lim1+'{message,comments'+lim+'{message,from{name'+pic+'}}}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['posts']['data']:
        data = {
            'message': d['message'],
            'data': [],
        }
        for dd in d['comments']['data']:
            data['data'].append({'name':dd['from']['name'], 'pic': dd['from']['picture']['data']['url'], 'message':dd['message']})
        
        ret.append(data)
        
    return {'name': name, 'pic': picture, 'data':ret}

def getMyPostShare(graph):
    event = graph.request('me?fields=name,posts'+lim1+'{message,shares}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['posts']['data']:
        data = {
            'message': d['message'],
            'count':d['shares']['counts']
        }
        ret.append(data)
        
    return {'name': name, 'pic': picture, 'data':ret}

def getMyLocation(graph):
    event = graph.request('me?fields=name,tagged_places'+lim1+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['tagged_places']['data']:
        data = {
            'message': d['place']['name'],
            'time': d['created_time']
        }
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}
    
def getMyPage(graph):
    event = graph.request('me?fields=name,likes'+lim+'{name'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['likes']['data']:
        data = {
            'name': d['name'],
            'pic': d['picture']['data']['url']
        }
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}
    
def getMyPageLike(graph):
    event = graph.request('me?fields=name,likes'+lim+'{name,fan_count'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['likes']['data']:
        data = {
            'name': d['name'],
            'pic': d['picture']['data']['url'],
            'count': d['fan_count']
        }
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}
    
def getMyPagePost(graph):
    event = graph.request('me?fields=name,likes'+lim+'{name,posts'+lim1+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for d in event['likes']['data']:
        posts = []
        for p in d['posts']['data']:
            try:
                data = {
                    'message': p['message'],
                    'time': p['created_time']
                }
                posts.append(data)
            except:
                posts = posts
        data = {
            'name': d['name'],
            'pic': d['picture']['data']['url'],
            'message': posts
        }
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

In [5]:
#Image(url=getMyPostComment(graph)['data'][0]['comments'][0]['pic'])
#Image(url=getMyPostComment(graph)['data'][0]['comments']['pic'])

In [6]:
#me?fields=friends{picture.height(720).width(720){url}}
#name, pic
def getFriend(graph):
    event = graph.request('me'+'?fields=name,friends'+lim+'{name'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url']
        }
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

#name, pic, device
def getFriendDevice(graph):
    event = graph.request('me'+'?fields=name,friends'+lim+'{name,devices.limit(1)'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
        }
        try:
            data['count']=friend['devices'][:1][0]['os']
        except:
            data['count']="อะไรไม่รู้ เขาไม่บอก"
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

#name, pic, post{msg, time, pic(ถ้ามี)}
def getFriendPost(graph):
    event = graph.request('me?fields=name,friends'+lim+'{name,posts'+lim1+''+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']    
    ret = []
    #print(event['friends']['data'][0]['posts'])
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
            'data' : []
        }
        try:
            for post in friend['posts']['data']:
                try:
                    a = {'message':post['message'], 'time':post['created_time']}
                    data['data'].append(a)
                except:
                    data = data
        except:
            ret = ret
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

#name, pic, book{name,pic}
def getFriendBook(graph):
    event = graph.request('me?fields=name,friends'+lim+'{name,books'+lim1+'{name'+pic+'}'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
            'data' : []
        }
        try:
            for book in friend['books']['data']:
                a = {'name':book['name'], 'pic':book['picture']['data']['url']}
                data['data'].append(a)
        except:
            data = data
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

#name, pic, friend{name,pic}
def getFriendFriend(graph):
    event = graph.request('me?fields=name,friends'+lim1+'{name,friends'+lim+'{name'+pic+'}'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
            'data' : []
        }
        try:
            for f in friend['friends']['data']:
                a = {'name':f['name'], 'pic':f['picture']['data']['url']}
                data['data'].append(a)
        except:
            data = data
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}
 
def getFriendLocation(graph):
    event = graph.request('me?fields=name,friends'+lim+'{name,tagged_places'+lim1+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']    
    ret = []
    #print(event['friends']['data'][0]['posts'])
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
            'data' : []
        }
        try:
            for post in friend['tagged_places']['data']:
                try:
                    a = {'message':post['place']['name'], 'time':post['created_time']}
                    data['data'].append(a)
                except:
                    data = data
        except:
            ret = ret
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}
    
#name, pic, page{name,pic}
def getFriendLike(graph):
    event = graph.request('me?fields=name,friends'+lim+'{name,likes'+lim1+'{name'+pic+'}'+pic+'}'+pic)
    name = event['name']
    picture = event['picture']['data']['url']
    ret = []
    for friend in event['friends']['data']:
        data = {
            'name': friend['name'],
            'pic': friend['picture']['data']['url'],
            'data' : []
        }
        try:
            for page in friend['likes']['data']:
                a = {'name':page['name'], 'pic':page['picture']['data']['url']}
                data['data'].append(a)
        except:
            data = data
        ret.append(data)
    return {'name': name, 'pic': picture, 'data':ret}

In [ ]:
from flask import Flask
from flask import jsonify
from flask import request
# from gtts import gTTS
# from pygame import mixer # Load the required library

app = Flask(__name__)
 
@app.route("/")
def hello():
    return "Hello World!"

def chk(keyword, result):
    idx = 0
    for word in keyword:
        idx = result.find(word,idx)
        if(idx==-1):
            return False
    return True

@app.route('/a')
def a():
    x = request.args.get('x')
    x =  str(x)
    tts = gTTS(text=str(x),lang='th') # text คือ ข้อความ lang คือ รหัสภาษา
    tts.save('hello-thai'+x+'.mp3')
    mixer.init()
    mixer.music.load('hello-thai'+x+'.mp3')
    mixer.music.play()
    return x

@app.route('/asr')
def asr():
    
    x = request.args.get('x')
    x = str(x)
    keywords = [['เรา', 'โพสต์'], 
                ['เรา', 'เช็คอิน'], 
                ['ล่าสุด', 'ยังไง'], 
                ['ล่าสุด', 'ถูกใจ'], 
                ['ล่าสุด', 'แชร์'],
                ['ล่าสุด', 'ความคิดเห็น'], 
                ['ขอ', 'เพื่อน', 'เรา'], 
                ['อะไร', 'เล่น', 'เฟซ'], 
                ['เพื่อน', 'ของ', 'เพื่อน'],
                ['หนังสือ'], 
                ['เพื่ื่อน', 'โพสต์'], 
                ['เพื่อน', 'เช็คอิน'], 
                ['เพื่อน', 'เพจ'], 
                ['ถูกใจ', 'เพจ'],
                ['เพจ', 'โพสต์'], 
                ['เพจ', 'เรา', 'ชอบ'] ]

    ret = dict()
    if chk(keywords[0], result):
        ret = getMyPost(graph)
        ret['type'] = 1
        script = 'โพสต์ล่าสุดของคุณ คือ ' + ret['data'][0]['message']
        ret['script'] = script
        
        
    elif chk(keywords[1], result):
        ret = getMyLocation(graph)
        ret['type'] = 1
        script = 'คุณเช็คอินล่าสุด ที่ ' + ret['data'][0]['message']
        ret['script'] = script
        
    elif chk(keywords[3], result):
        ret = getMyPostLike(graph)
        ret['type'] = 2
        script = 'โพสต์ล่าสุดของคุณมีคนถูกใจ ' + ret['data'][0]['count'] + ' คน'
        ret['script'] = script
        
    elif chk(keywords[4], result):
        ret = getMyPostShare(graph) #['ล่าสุด', 'แชร์']
        ret['type'] = 2
        script = 'โพสต์ล่าสุดของคุณมีคนแชร์ ' + ret['data'][0]['count'] + ' คน'
        ret['script'] = script
        
    elif chk(keywords[5], result):
        ret = getPostComment(graph) #['ล่าสุด', 'ความคิดเห็น']
        ret['type'] = 3
        
        script = ""
        for x in ret['data']:
            try:
                tmp = ' ' + x['name'] + 'แสดงความคิดเห็นว่า ' + x['message']
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[6], result):
        ret = getFriend(graph)
        ret['type'] = 4
        script = "เพื่อนของคุณ ได้แก่"
        for x in ret['data']:
            try:
                tmp = ' ' + x['name']
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[7], result):
        ret = getFriendDevice(graph)
        ret['type'] = 5
        script = ""
        for x in ret['data']:
            try:
                stmp = x['name'] + ' ใช้ ' + x['count']
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[8], result):
        ret = getFriendFriend(graph)
        ret['type'] = 6
        script = "เพื่อนของ " + ret['data'][0]['name'] + ' ได้แก่ '
        for x in ret['data'][0]['data']:
            try:
                tmp = x['name'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[9], result):
        ret = getFriendBook(graph)
        ret['type'] = 6
        script = ''
        for x in ret['data']:
            try:
                tmp = x['name'] + ' มีหนังสือที่ชอบคือ ' + x['data'][0]['name'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[10], result):
        ret = getFriendPost(graph)
        ret['type'] = 7
        script = ''
        for x in ret['data']:
            try:
                tmp =  x['name'] + ' โพสต์ว่า ' + x['data'][0]['message'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[11], result):
        ret = getFriendLocation(graph)
        ret['type'] = 7
        script = ''
        for x in ret['data']:
            try:
                tmp = x['name'] + ' เช็คอินที่ ' + x['data'][0]['message'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[12], result):
        ret = getFriendLike(graph)
        ret['type'] = 6
        script = ''
        for x in ret['data']:
            try:
                tmp = x['name'] + ' ชอบเพจ ' + x['data'][0]['name'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[13], result):
        ret = getMyPageLike(graph)
        ret['type'] = 5
        script = ''
        for x in ret['data']:
            try:
                tmp = 'เพจ ' + x['name'] + ' มีคนถูกใจ ' + x['data'][0]['count'] + ' คน '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[14], result):
        ret = getMyPagePost(graph)
        ret['type'] = 7
        script = ''
        for x in ret['data']:
            try:
                tmp = 'เพจ ' + x['name'] + ' โพสต์ว่า ' + x['data'][0]['message'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    elif chk(keywords[15], result):
        ret = getMyPage(graph)
        ret['type'] = 4
        script = 'เพจที่คุณชอบ ได้แก่ '
        for x in ret['data']:
            try:
                tmp = 'เพจ ' + x['name'] + ' '
                script = script + tmp
            except:
                script = script
        ret['script'] = script
        
    else:
        ret['data'] = "ขออีกที"
        ret['type'] = 99
    
    return ret
 
if __name__ == "__main__":
    app.run()

In [ ]:
from IPython.display import Image
def showimg(URL):
    Image(url=URL) 

In [36]:
import subprocess
def runclient(filename):
    result = subprocess.run(['python','client.py',filename], stdout=subprocess.PIPE)
    return result.stdout.decode("utf-8")

In [27]:
print(runclient(1).decode("utf-8"))

.
..
.DS_Store
.ipynb_checkpoints
Untitled.ipynb
a
asr.ipynb
asrbytm.ipynb
b
backend
c
node
test



In [46]:
"abcde".find(['a','b'])

TypeError: must be str, not list

In [52]:
"abcdeaaa".find('a',8)

-1

In [ ]:
"abcde".find

In [53]:
keywords = [['เรา', 'โพสต์'], ['เรา', 'เช็คอิน'], ['ล่าสุด', 'ยังไง'], ['ล่าสุด', 'ถูกใจ'], ['ล่าสุด', 'แชร์'],\
               ['ล่าสุด', 'ความคิดเห็น'], ['ขอ', 'เพื่อน', 'เรา'], ['อะไร', 'เล่น', 'เฟซ'], ['เพื่อน', 'ของ', 'เพื่อน'],\
               ['หนังสือ'], ['เพื่ื่อน', 'โพสต์'], ['เพื่อน', 'เช็คอิน'], ['เพื่อน', 'เพจ'], ['ถูกใจ', 'เพจ'],\
               ['เพจ', 'โพสต์'], ['เพจ', 'เรา', 'ชอบ'] ]
len(keywords)

16

In [54]:
ret

NameError: name 'ret' is not defined

In [9]:
x

NameError: name 'x' is not defined